In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import csv
import time
import librosa

import warnings
import pandas as pd
from models import *
from utils import *
import torch
import torchaudio
import numpy as np
import torch.optim as optim

warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
%%time 

limit = 80
params = {'batch_size': 20,
          'shuffle': True,
          'num_workers': 10,
          'drop_last': True}


tokenizer = tokenizer_librispeech(limit=800)
training_set = LibriSpeechDataset(limit=limit, tokenizer=tokenizer, n_frames=600)
my_dataloader = data.DataLoader(training_set, **params)

steps_per_epoch = len(training_set) // params['batch_size']

vocab_tar_size = len(tokenizer.word_index)
embedding_dim_target = 100

 ===== They are 800 transcriptions in the dataset. ===== 
CPU times: user 521 ms, sys: 76.5 ms, total: 598 ms
Wall time: 597 ms


In [ ]:
# 'bahdanau_basic'
# 'bahdanau_audio'
# 'luong_dot'
# 'luong_concat'
# 'luong_general'
# 'super_head'

In [17]:
encoder = EncoderCONV2DRNN(batch_size=params['batch_size'], device=device, hidden_size=64).to(device)
decoder = DecoderATTRNN1(vocab_tar_size, embedding_dim_target, dec_units=64, batch_sz=params['batch_size'], 
                         hidden_size=64, method='bahdanau_basic').to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [18]:
# Train the model
global_trainer(1, my_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer,
                                    criterion, device, params['batch_size'], tokenizer )

 ====== ====== ====== ====== ====== ======
      The model has 1809955 parameters
 ====== ====== ====== ====== ====== ======


Epoch 1: 100%|████████████████████| 4/4 [00:19<00:00,  4.87s/it, Loss 8.1813]

Time taken for the training 0.0054092 hours



In [19]:
mfccs, references = training_set[1]
references = [tokenizer.index_word[ind] for ind in references.numpy()]
evaluate(mfccs.unsqueeze(0), references, 40, encoder, decoder, targ_lang=tokenizer, 
          device=device)

Input: ['<start>', 'what', 'did', 'you', 'do', 'next', 'i', 'awakened', 'with', 'a', 'sudden', 'start', 'just', 'before', 'six', "o'clock", 'i', 'had', 'not', 'set', 'an', 'alarm', 'though', 'i', 'wanted', 'to', 'get', 'up', 'early', 'to', 'do', 'a', 'little', 'repair', 'job', 'i', 'had', 'promised', 'for', 'early']


Predicted translation: ['<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>', '<padding>']


Bleu score: 0
